In [ ]:
# %rm -rf phase2_module
# !git clone https://github.com/LgDNet/phase2_module.git
# !cd phase2_module && make copy

In [1]:
cd ..

/home/elicer


/home/elicer/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from phase2_module.src.utils.dataset import Data
from phase2_module.src.models.classification.models import Model
from phase2_module.src.base import BasePiepline
# from phase2_module.src.hyper_parameters.params_optimization import xgboost_opt, set_params_optimization_data
from phase2_module.src.utils.set_seed import seed_everything
from phase2_module.src.utils.top_score_instance import check_the_score
from phase2_module.src.encoder.encoders import Encode

from phase2_module.src.utils.manage_pkl_files import PickleManager as pkl_manager
from phase2_module.src import preprocess
seed_everything()  # NOTE: default 42
_model = Model()

In [3]:
import pandas as pd
import numpy as np
import re, pickle
import warnings
from tqdm import tqdm
warnings.filterwarnings(action='ignore')
def set_up(mode= None):
    pd.set_option('display.max_columns', mode)
set_up()

# Preprocessing

In [4]:
basic = preprocess.Basic
country = preprocess.Country
customer = preprocess.Customer
inquiry = preprocess.Inquiry
product_category = preprocess.ProductCategory
convert_ratio = preprocess.Convert_ratio
customer2 = preprocess.Customer2
category = preprocess.Categorization

# Main

In [5]:
pkls = pkl_manager.map(pkl_manager.loads, pkl_manager.metadata_directory)

runtime_instance = {
    "Train": Data.train,
    "Test": Data.test,
    "Instances":[
        {"instance": basic, "method": ['drop_duplicated'], "params": False},
        {"instance": country, "method": ['apply'], "params": pkls.get("country")},
        {"instance": inquiry, "method": ["apply"], "params": False},
        {"instance": product_category, "method": ["product_categories", 'fill_odds'], "params": {**pkls.get("product_category"), **pkls.get("product_subcategory")}},
        {"instance": customer, "method": ["apply"], "params": pkls.get("customer")},
        {"instance": convert_ratio, "method": ["lead_owner"], "params": {**pkls.get("lead_owner"), **pkls.get("customer_idx")}},
#         {"instance": basic, "method": ['customer_idx_merge_enterprise'], "params": False},
        {"instance": customer2, "method": ['seniority_level', 'job_function', 'customer_type'], "params": pkls.get("customer2")},
        {"instance": category, "method": ['cat_customer_idx','customer_idx_merge_enterprise','historical_cnt_mean'], "params": False},
    ]
}

## preprocessing

In [19]:
df_train = preprocess.runner('Train',**runtime_instance)
df_test = preprocess.runner('Test',**runtime_instance)

In [21]:
cond = df_train['is_converted'] == True
true_num = df_train[cond]['lead_desc_length'].mean()
cond1 = df_train['lead_desc_length'] > true_num
df_train.loc[cond1,'less_timeline'] += 1

## encoding

In [24]:
encode = Encode()

In [25]:
exisiting_columns = [
                     'business_area','business_subarea','business_unit',
                        'enterprise',
#                         'response_corporate',#"customer_country", # country
#                         'customer_type','customer_job','customer_position', #customer
#                         'inquiry_type','expected_timeline', # inquiry
#                         'product_subcategory','product_category','product_modelname' # product_category
                       ] 
preprocess_columns = [
                     'country','continent', # country
                     'customer_type2','job_function','seniority_level', # customer
                     'new_expected_timeline', 'new_inquiry_type',# inquiry
                     'category_2','category_3','category_1',
                     'new_customer_type1','new_customer_type2' # product_category
#                      'test2'
                     ]

label_encode_columns = exisiting_columns + preprocess_columns

In [26]:
# 결측치 채우기
df_train = preprocess.fillna(df_train)
df_test = preprocess.fillna(df_test)

In [27]:
# # one-hot encoding
# onehot_encode_columns = ['category_1']
# df_train = encode.onehot_encoder(df_train,onehot_encode_columns)
# df_test = encode.onehot_encoder(df_test,onehot_encode_columns)

In [28]:
# label encoding
encode.set_up() # 전역 변수 초기화
df_train = encode.label_encoder(df_train,label_encode_columns)
df_test = encode.label_encoder(df_test,label_encode_columns)

# Modeling

## drop column
모델링 전이 column을 drop하면서 다루기가 쉽다고 판단하였음

In [29]:
drop_col = [
            'id_strategic_ver','it_strategic_ver','idit_strategic_ver','ver_pro','ver_cus',
            'customer_country.1',
            "customer_country",#'response_corporate' # country
            'customer_type','customer_job','customer_position', # customer
            'inquiry_type', 'expected_timeline', #inquiry
            'product_subcategory','product_category','product_modelname', # product_category
#             'cate_is_nan', # product_category
    
#             'com_reg_ver_win_rate','idit_strategic_ver','ver_cus',
#             'ver_pro','ver_win_rate_x','ver_win_ratio_per_bu',
            'customer_idx',
            'lead_owner','historial_existing_mean','response_corporate',#'new_customer_idx'
            'customer_type2',
#             'customer_idx_converted_ratio', # ratio
            #'lead_owner', 'lead_owner_converted_ratio', 'lead_owner_customer_idx_mean', # lead_owner
            ]
df_train.drop(drop_col,axis = 1, inplace = True)
df_test.drop(drop_col,axis = 1, inplace = True)

## Scaler

In [30]:
import numpy as np
# log 변환
df_train['lead_desc_length'] = np.log1p(df_train['lead_desc_length'])
df_test['lead_desc_length'] = np.log1p(df_test['lead_desc_length'])

In [31]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train[['lead_owner_converted_ratio']] = scaler.fit_transform(df_train[['lead_owner_converted_ratio']])
df_test[['lead_owner_converted_ratio']] = scaler.transform(df_test[['lead_owner_converted_ratio']])

## type

In [ ]:
df_train['new_customer_idx'] = df_train['new_customer_idx'].astype(int)
df_test['new_customer_idx'] = df_test['new_customer_idx'].astype(int)

## train

In [32]:
import math
from six.moves import xrange

class LoglossObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)
        
        result = []
        for index in range(len(targets)):
            e = np.exp(approxes[index])
            p = e / (1 + e)
            der1 = targets[index] - p
            der2 = -p * (1 - p)
            

            if (targets[index] == 0.0) & (p<0.5):
                #( True Negative )
                der1 = (p)*(targets[index] - p)
                der2 = (1-2*p)*(-p * (1 - p))
                
            elif (targets[index] == 0.0) & (p>=0.5):
                # 타겟값이 노말 유저인데,fraud 라고 예측한 경우 ( False Positive )
                der1 = 3*(p)*(targets[index] - p)
                der2 = 3*(1-2*p)*(-p * (1 - p))

            elif (targets[index] > 0.0) :
                # 타겟 값이 fraud 이라면 p값이 높을수록 (정답에 가까울수록) 페널티가 적도록 세팅. 추가로 3배 페널티
                der1 = 3*(1-p)*(targets[index] - p)
                der2 = 3*(2*p-1)*(-p * (1 - p))
                
            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))
        return result

In [33]:
model = _model.catboost
model.set_params(iterations=5000,
                depth=12,
                thread_count=3,
                learning_rate=1e-3,
                loss_function=LoglossObjective(),
                eval_metric='PRAUC',
                early_stopping_rounds=400,
                verbose=1000)

In [154]:
dict_ = {'criterion': 'log_loss', 'max_depth': 20, 'min_samples_split': 7, 'min_samples_leaf': 2, 'random_state': 42}
model = _model.decision_tree
model.set_params(**dict_)

DecisionTreeClassifier(criterion='log_loss', max_depth=20, min_samples_leaf=2,
                       min_samples_split=7, random_state=42)

In [238]:
model = _model.xgboost
model.set_params(**{'n_estimators': 2393,
 'max_depth': 16,
 'min_child_weight': 2,
 'learning_rate': 0.06783336462585174,
 'subsample': 0.9511145375386896,
 'gamma': 0.16846211770550093,
 'colsample_bytree': 0.6662896237544925,
 'colsample_bylevel': 0.8781042717330898,
 'colsample_bynode': 0.741271661588135,
'random_state':42})
# model.set_params(random_state=42)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.8781042717330898,
              colsample_bynode=0.741271661588135,
              colsample_bytree=0.6662896237544925, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.16846211770550093,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.06783336462585174,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=16, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=2393, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [18]:
model = _model.lightgbm
model.set_params(random_state = 42)

LGBMClassifier(n_estimators=150, random_state=42)

In [151]:
X = df_train.drop(["is_converted"],axis = 1)
Y = df_train['is_converted']
# Y = np.where(Y == False, 0, 1) # catboost

In [20]:
# oversampling
from imblearn.over_sampling import SMOTE, ADASYN
adasyn = SMOTE(sampling_strategy=0.2,random_state=42)
X, Y = adasyn.fit_resample(X, Y)

In [155]:
def score_print(result_dict):
    for name, score_list in result_dict.items():
        print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

In [287]:
from collections import defaultdict
from sklearn.model_selection import StratifiedKFold
stratkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_result, valid_result = defaultdict(list), defaultdict(list) # result 리스트
models = [] # 모델을 저장할 리스트

# k-fold
for idx,(train_idx, test_idx) in tqdm(enumerate(stratkfold.split(X,Y))):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]

    # 모델 선언
    dict_ = {'criterion': 'log_loss', 'max_depth': 20, 'min_samples_split': 7, 'min_samples_leaf': 2, 'random_state': 42}
    model = _model.decision_tree
    model.set_params(**dict_)
    
    # 모델 훈련
    model.fit(x_train, y_train)
    models.append(model) # 훈련된 모델을 리스트에 추가
    train_pred, valid_pred = models[idx].predict(x_train), models[idx].predict(x_test)
    
    # store result
    for name, score in check_the_score(train_pred, y_train).items(): # train
        train_result[name].append(score)
    for name, score in check_the_score(valid_pred, y_test).items():  # valid
        valid_result[name].append(score)
# print score
print('----[K-Fold Train Score]-----')
score_print(train_result)
print('----[K-Fold Validation Score]-----')
score_print(valid_result)

5it [00:19,  3.83s/it]

----[K-Fold Train Score]-----
f1 score : 0.9884 / STD: (+/- 0.0012)
precision score : 0.9824 / STD: (+/- 0.0018)
recall score : 0.9946 / STD: (+/- 0.0007)
roc_auc_score score : 0.9965 / STD: (+/- 0.0004)
----[K-Fold Validation Score]-----
f1 score : 0.8336 / STD: (+/- 0.0103)
precision score : 0.7801 / STD: (+/- 0.0178)
recall score : 0.8952 / STD: (+/- 0.0054)
roc_auc_score score : 0.9378 / STD: (+/- 0.0028)


In [257]:
# threshold valid
threshold = 0.6
valid_result = defaultdict(list) # result 리스트
for idx, (_, test_idx) in tqdm(enumerate(stratkfold.split(X,Y))):
    x_test, y_test = X.iloc[test_idx], Y[test_idx]

    y_valid_pred = models[idx].predict_proba(x_test)[:,0]
    y_valid_pred = np.where(y_valid_pred >= threshold, False, True)
    
    for name, score in check_the_score(y_valid_pred, y_test).items():  # valid
        valid_result[name].append(score)
        
# print score
print('----[K-Fold Validation Score]-----')
score_print(valid_result)

5it [00:00, 31.41it/s]

----[K-Fold Validation Score]-----
f1 score : 0.7709 / STD: (+/- 0.0052)
precision score : 0.7913 / STD: (+/- 0.0097)
recall score : 0.7517 / STD: (+/- 0.0143)
roc_auc_score score : 0.8664 / STD: (+/- 0.0069)


In [295]:
# feature importance
feature_importances = model.feature_importances_
feature_names = x_train.columns
feature_importance_dict = dict(zip(feature_names, feature_importances))
sorted_importance_dict = {k: v for k, v in sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True)}
for feature_name, importance in sorted_importance_dict.items():
    print(f"{feature_name}: {importance}")

lead_owner_converted_ratio: 0.19115176796913147
new_customer_idx: 0.1212412491440773
business_subarea: 0.05201156437397003
ver_win_ratio_per_bu: 0.04938187822699547
ver_win_rate_x: 0.0472320094704628
com_reg_ver_win_rate: 0.0448000431060791
new_customer_type2: 0.04023168981075287
business_area: 0.0362914502620697
historical_existing_cnt: 0.03588666021823883
business_unit: 0.033222656697034836
new_customer_type1: 0.030143167823553085
country: 0.02935674600303173
continent: 0.028711553663015366
category_1: 0.027089107781648636
customer_interest: 0.025827668607234955
category_2: 0.02473408356308937
less_timeline: 0.024620896205306053
bant_submit: 0.023783709853887558
lead_desc_length: 0.02234685607254505
category_3: 0.021855270490050316
new_inquiry_type: 0.020726200193166733
new_expected_timeline: 0.020184090360999107
job_function: 0.019330579787492752
seniority_level: 0.017157353460788727
enterprise: 0.012681732885539532


## inference

In [288]:
test_drop = df_test.drop(['id','is_converted'],axis = 1)

In [292]:
# voting threshold
predicts = []
threshold = 0.9
for model in models:
    pred_proba = model.predict_proba(test_drop)[:,0]
    pred = np.where(pred_proba >= threshold, False, True)
    predicts.append(pred)

In [293]:
for i in predicts:
    print(sum(i))

1237
1226
1214
1227
1212


In [294]:
num_threshold = 1
pred = np.where( np.sum(predicts,axis = 0) >= num_threshold, 1, 0)
pred = np.where(pred == 0, False, True)
sum(pred)

1446

In [168]:
# difference
pred3 = pred  #pred1 718 | pred2 1142 | pred3 820
print(sum(pred1))

In [185]:
su = 0
for i,j in zip(pred3, pred1):
    if i != j:
        su +=1
su

249

## submission

In [232]:
df_test['is_converted'] = pred
df_test.to_csv("submission.csv",index = False)

In [233]:
sum(pred)

1390